In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Cyclist_env_RDA import cyclist_env
from time import time
from scipy.linalg import dft
import torch
import numpy as np
import torch.nn.functional as F
from scipy.optimize import linear_sum_assignment
from tqdm import tqdm

In [30]:
# CFAR検出器の実装

def ca_cfar_2d(x_db, n_train=(1,7), n_guard=(1,5), pfa=1e-6, convert_from_db=False):
    """
    x_db: (B,1,H,W) input map (dB or linear)
    n_train: (th, tw) training half window size
    n_guard: (gh, gw) guard half window size
    """
    if convert_from_db:
        x = 10 ** (x_db / 10.0)  # dB -> linear
    else:
        x = x_db

    B, C, H, W = x.shape
    th, tw = n_train
    gh, gw = n_guard

    # Total kernel size
    kh = 2 * (th + gh) + 1
    kw = 2 * (tw + gw) + 1

    # Make training mask
    kernel = torch.ones((1,1,kh,kw), device=x.device)
    kernel[:, :, th-gh:th+gh+1, tw-gw:tw+gw+1] = 0  # guard + CUT = 0

    N_train = kernel.sum().item()
    alpha = N_train * (pfa ** (-1.0 / N_train) - 1.0)

    # Convolution = training sum
    x_pad = F.pad(x, (kw//2, kw//2, kh//2, kh//2), mode="reflect")
    train_sum = F.conv2d(x_pad.double(), kernel.double())
    noise_est = train_sum / N_train

    threshold = alpha * noise_est
    detections = x > threshold
    return detections, threshold, noise_est

# 変更前: tar_thres = -25
# 変更後: tar_thres = -35  (または -30)
def thresholder(radar, idx_est, tar_thres = -25, cyvethres = 2.638):
    cy_idx, ve_idx = [], []
    radar = 20*torch.log10(radar)
    for i in range(len(idx_est)):
        rangeofinterest = radar[:5, max(0, idx_est[i]-5):min(idx_est[i]+5, 589)].flatten()
        top2 = rangeofinterest.topk(2).values.mean()
        if top2 > tar_thres:
            cy_idx.append(idx_est[i])
            ve_idx.append(idx_est[i]) 

    cy_idx_cleans = cleansing(cy_idx)
    ve_idx_cleans = cleansing(ve_idx)

    return cy_idx_cleans, ve_idx_cleans
    
def cleansing(idx, tol = 5):
    if idx :
        res = [idx[0]]
        for i in range(1,len(idx)):
            if idx[i]<=idx[i-1]+5:
                continue
            res.append(idx[i])
        return res
    else: return []

In [31]:
def stevec(N_ant, angle):
    resp = (np.arange(N_ant)-N_ant*0.5+0.5).reshape([-1,1])
    resp = np.exp(1j*resp*np.pi*np.sin(angle))
    return np.matrix(resp)

In [44]:
# csvファイルの読み込み機能を関数化
def syntax_check(num, idx, y_value, velocity_value, x_pos):
    idx_list = np.zeros(num, dtype=np.int32)
    position, velocity = np.zeros((num, 3)), np.zeros((num, 3))
    i=0
    s = set()
    while i < num:
        # 初期値における距離間隔を保証するための処理
        if idx+10 in s or idx-10 in s or idx in s: continue
        for j in range(-20,20): s.add(idx+j)
        
        idx_list[i] = idx
        position[i] = np.array([x_pos[idx], y_value, 1])
        # v_cy[i] = np.array([np.random.rand(1)[0]*5.56, 0, 0])
        velocity[i] = np.array([velocity_value, 0, 0])
        
        i+=1
    return idx_list, position, velocity

def get_snapshot_data(cy_idx_target, ve_idx_target,x_pos, p_bs, tx):
    # Cyclistについての設定
    num_cy = 1
    cy_idx = np.zeros(num_cy, dtype=np.int32)
    p_cy, v_cy = np.zeros((num_cy, 3)), np.zeros((num_cy, 3))
    # 範囲外アクセスを防ぐための処理
    cy_idx, p_cy, v_cy = syntax_check(num_cy, cy_idx_target, 15, 5, x_pos)
    n_cy = len(p_cy)
    
    # Vehicleについての設定
    num_ve = 1
    ve_idx = np.zeros(num_ve, dtype=np.int32)
    p_ve, v_ve = np.zeros((num_ve, 3)), np.zeros((num_ve, 3))
    # 範囲外アクセスを防ぐための処理
    ve_idx, p_ve, v_ve = syntax_check(num_ve, ve_idx_target, 7.5, 10, x_pos)
    n_ve = len(p_ve)
    # ramppostについての設定
    num_rp = 0
    p_rp, v_rp = np.zeros((num_rp, 3)), np.zeros((num_rp, 3))
    n_rp = len(p_rp)
    
    # csvファイルの読み込み
    cy_idx += 1
    ve_idx += 1

    P_rx_cy, tstemp_rx_cy, phase_cy, P_rx_ve, tstemp_rx_ve, phase_ve, P_rx_rp, tstemp_rx_rp, phase_rp = [], [], [], [], [], [], [], [], []
    for i in range(len(cy_idx)):
        df = pd.read_csv("./ped/0.5nano/complex-impulse-response-Run{:04d}-Sensor_0_Tx_0_to_Rx_0.csv".format(cy_idx[i]))
        tstemp_rx_cy.append(np.ceil((df["Time (s)"]/Tc).values[3:]))
        P_rx_cy.append(df["| Total Complex Impulse Response total | (W)"].values[3:])
        phase_cy.append(df['Phase( Total Complex Impulse Response total ) (rad)'].values[3:])
    for i in range(len(ve_idx)):
        df = pd.read_csv("./vehicle/0.5nano/complex-impulse-response-Run{:04d}-Sensor_0_Tx_0_to_Rx_0.csv".format(ve_idx[i]))
        tstemp_rx_ve.append(np.ceil((df["Time (s)"]/Tc).values[3:]))
        P_rx_ve.append(df["| Total Complex Impulse Response total | (W)"].values[3:])
        phase_ve.append(df['Phase( Total Complex Impulse Response total ) (rad)'].values[3:])
    for i in range(1,4):
        df = pd.read_csv("./Ramposts/"+str(i)+"/complex-impulse-response-Run0001-Sensor_0_Tx_0_to_Rx_0.csv")
        tstemp_rx_rp.append(np.ceil((df["Time (s)"]/Tc).values[5:]))
        P_rx_rp.append(df["| Total Complex Impulse Response total | (W)"].values[5:])
        phase_rp.append(df['Phase( Total Complex Impulse Response total ) (rad)'].values[5:])
        
    phys_quantities = env.phys_quantities(p_bs, p_cy, v_cy, n_cy, p_ve, v_ve, n_ve, p_rp, v_rp, n_rp)
    
    
    ######### We need some functions here. it is like this. read the csv. this outputs the time stemp and corresponding rx value.
    # P_rx_cy_dB = -114
    #P_rx_cy_dB = -111
    #P_rx_ve_dB = -114
    #P_rx_cy_dB = 0
    #P_rx_ve_dB = 0
    
    P_N_dB = -81 # target noise power when using 2GHz BW

    sym_duration = rx_sample*Tc

    Y = env.rx_multiple(tx, P_rx_cy, tstemp_rx_cy, phase_cy, P_rx_ve, tstemp_rx_ve, phase_ve, P_rx_rp, tstemp_rx_rp, phase_rp, P_N_dB, sym_duration, N_trans)
    #print("Cyclelist and Vehicle positions:")
    #print(p_cy)
    #print(p_ve)
    return Y, phys_quantities

#x_pos = list(np.arange(-450, -149)/10)
#p_bs = np.array([250, -18, 50])
#Y, phys_quantities = get_snapshot_data(100,0,x_pos,p_bs)

In [33]:
def MUSIC_method(Y, N_ant):
    Y_music = np.mean(Y, axis=0)
    R_yy = Y_music@np.conjugate(np.transpose(Y_music))
    # print(np.linalg.eig(R_yy)[0])
    U = np.linalg.eig(R_yy)[1][:,5:]

    resp = []
    argm = (np.arange(1800)-900)/100
    for val in argm:
        stv = stevec(N_ant, val*np.pi/180)
        p = stv.T@U
        pp = p*np.conjugate(np.transpose(p))
        resp.append(1/(np.abs(pp)**2).A1)

    M = max(resp)
    est_ang = []
    
    a, b = resp[0][0], resp[1][0]
    for i in range(1,len(argm)-1):
        c = resp[i+1][0]
        if a < b and b > c and b > 0.2 * M:
            est_ang.append(argm[i])
        a, b = b, c
    return est_ang

#angle_list = MUSIC_method(Y)
#print("Estimated angles:", angle_list)

In [40]:
p_bs = np.array([250, -18, 50])

def idx_to_xy(range_idx, angle_deg):
    """
    range_idx: cy_idxs_pred の値 (例: 496)
    angle_deg: MUSIC法で推定した角度 [度] (例: est_ang[0])
    """
    offset = 3411
    # (1) 距離の計算 [m]
    # サンプル番号 = インデックス + オフセット
    sample_idx = range_idx + offset
    # 往復時間 = サンプル番号 * Tc
    # 距離 = 往復時間 * c / 2
    R = (sample_idx * Tc * env.c) / 2

    # (2) 角度の計算 [rad]
    # 推定角度をラジアンに変換し、基準角度(refangle)を足す
    # anglecalの逆演算: cy_ang = arctan(...) - refangle なので
    # arctan(...) = cy_ang + refangle
    theta_world = np.deg2rad(angle_deg) + refangle

    # (3) XY座標の計算 [m]
    # 基地局(p_bs)からの相対位置
    # cy_denum (dx) = R * cos(theta)
    # cy_nu (dy) = R * sin(theta)
    dx = R * np.cos(theta_world)
    dy = R * np.sin(theta_world)

    # 世界座標（絶対座標）に変換
    # cy_denum = bs[0] - cy[0]  =>  cy[0] = bs[0] - dx
    # cy_nu    = cy[1] - bs[1]  =>  cy[1] = bs[1] + dy
    x = p_bs[0] - dx
    y = p_bs[1] + dy

    return x, y, R

In [50]:
def DFT (N_trans, N_ant, rx_sample, Y, tx, N_sample, est_ang):
    Dopp_dft = dft(N_trans)/np.sqrt(N_trans)
    Y_dft = np.zeros((N_trans, N_ant, rx_sample), dtype=np.complex128)
    for i in range(N_ant):
        Y_dft[:,i,:] = Dopp_dft@Y[:,i,:]
    val, res, rdresp = 0, [], np.zeros((N_trans,590), dtype=np.complex128)
    Q = 1

    # 結果を保存するリスト
    detected_objects = []
    
    P_range = np.arange(590)+3411
    #P_range = np.arange(150) + 3800
    for i in range(len(est_ang)):
        current_angle = est_ang[i]
        #  他の角度の結果と混ざらないように rdresp_single を使う
        rdresp_single = np.zeros((N_trans, 590), dtype=np.complex128)
        ang_rad = current_angle * np.pi / 180
        dup = np.ones((N_trans, 1, 1), dtype=np.complex128)
        
        for p in P_range: # レンジ方向のループ
            # ... (既存の信号生成・相関処理 tx, stevec など) ...
            X_cand = np.zeros((N_ant, rx_sample), dtype=np.complex128)
            last = min(p+N_sample, rx_sample)
            X_cand[:,p:last] += tx[:,:last-p]
            g = np.conjugate(stevec(N_ant, ang_rad))@np.conjugate(np.transpose(stevec(N_ant, ang_rad)))@X_cand
            g_mat = dup*np.array(g)
            metric = np.sum(np.multiply(np.conjugate(Y_dft), g_mat), axis=(1,2))
            
            # ここで rdresp ではなく rdresp_single に保存
            rdresp_single[:, p-3411] = metric
            
        # CFAR検出器の利用
        radar_in = torch.tensor(abs(rdresp_single)).unsqueeze(0).unsqueeze(0)
        detections, _, _ = ca_cfar_2d(radar_in, convert_from_db=False)
        idx_est = torch.nonzero(detections[0,0,0], as_tuple=True)[0].tolist()
        current_range_idxs, _ = thresholder(radar_in[0,0], idx_est)
        
        print("検出した距離：", current_range_idxs)
        
        # 座標変換と結果保存
        if len(current_range_idxs) > 0:
            print(f"角度 {current_angle:.2f} 度 のマップからの検出:")
            for r_idx in current_range_idxs:
                x_est, y_est, r_est = idx_to_xy(r_idx, current_angle)
                detected_objects.append([x_est, y_est])
                print(f"  -> Index: {r_idx}, Range: {r_est:.2f}m, X: {x_est:.2f}m, Y: {y_est:.2f}m")
        else:
            print(f"ターゲット不検出")
    return detected_objects

#Y, phys_quantities = get_snapshot_data(100,0)
#est_ang = MUSIC_method(Y)
#detected_objects = DFT(N_trans, N_ant, rx_sample, Y, tx, N_sample, est_ang)
#print("Detected objects (X, Y):", detected_objects)

In [52]:
###### symbol time & carrier frequency ######
T_symbol = 1.115 * 1e-6              # symbol duration, with CP time
T_OFDM = 1.0425 * 1e-6
f_carrier = 28 * 1e+9
Tc = 0.509*1e-9                      # sampling time

###### tx/rx ######
N_ant = 16                       # the number of antennas
BW = 1.966080e+9                      # chirp bandwidth
BW_sub = BW/N_ant
N_sample = int(np.floor(T_symbol/Tc))           # the number of samples of single chirp
offset = 4000
rx_sample = offset + N_sample



###### Radar setting ######

N_chirp = 100                    # the number of chirps
mu = BW_sub/T_symbol * 0.98
Q = 30
Phi = 0.3*np.pi

l_speed = 299792458

N_trans = 100

refangle = np.arctan(33/265)/2 # 基準角度の設定(cyclist_env_RDA.pyより)


env = cyclist_env(f_carrier, N_ant, BW, BW_sub, N_sample, rx_sample, Tc, mu, l_speed)
tx = env.tx()

# シミュレーション設定
#start_cy_idx = 300
#start_ve_idx = 3
start_cy_idx = 75
start_ve_idx = 50

#end_cy_idx = 300
#end_ve_idx = 3
end_cy_idx = 135
end_ve_idx = 150

cy_interval = 2
ve_interval = 5

cy_idx = start_cy_idx
ve_idx = start_ve_idx

measurement_history = []
frame_count = 0

# 繰り返し処理
while cy_idx <= end_cy_idx and ve_idx <= end_ve_idx:
    x_pos = list(np.arange(-450, -149)/10)
    p_bs = np.array([250, -18, 50])
    Y, phys_quantities = get_snapshot_data(cy_idx,ve_idx,x_pos,p_bs, tx)
    est_ang = MUSIC_method(Y, N_ant)
    print(est_ang)
    detected_objects = DFT(N_trans, N_ant, rx_sample, Y, tx, N_sample, est_ang)
    print("Detected objects (X, Y):", detected_objects)
    
    if len(detected_objects) > 0:
        for obj in detected_objects:
            measurement_history.append({
                "Time": frame_count,  # 時刻（フレーム番号）
                "X": obj[0],          # X座標 [m]
                "Y": obj[1]           # Y座標 [m]
            })
    else:
        # 何も検出されなかった場合も、時刻だけ記録したい場合は以下のようにNaNを入れる
        # (不要であればこのelseブロックは削除してください)
        measurement_history.append({
            "Time": frame_count,
            "X": np.nan,
            "Y": np.nan
        })
    
    cy_idx += cy_interval
    ve_idx += ve_interval
    frame_count += 1
    
# csvファイルに保存
# --- 6. CSVへの保存 ---
df_measurements = pd.DataFrame(measurement_history)

# 保存
csv_filename = "measurements.csv"
df_measurements.to_csv(csv_filename, index=False)

print("\nData generation complete.")
print(f"Saved '{csv_filename}'.")




[np.float64(1.5)]
検出した距離： []
ターゲット不検出
Detected objects (X, Y): []
[np.float64(1.53)]
検出した距離： [426]
角度 1.53 度 のマップからの検出:
  -> Index: 426, Range: 292.75m, X: -41.60m, Y: 7.92m
Detected objects (X, Y): [[np.float64(-41.60272195060003), np.float64(7.918160765374964)]]
[np.float64(1.53)]
検出した距離： [421]
角度 1.53 度 のマップからの検出:
  -> Index: 421, Range: 292.37m, X: -41.22m, Y: 7.88m
Detected objects (X, Y): [[np.float64(-41.222734040838986), np.float64(7.884386774281168)]]
[np.float64(1.53)]
検出した距離： [414]
角度 1.53 度 のマップからの検出:
  -> Index: 414, Range: 291.84m, X: -40.69m, Y: 7.84m
Detected objects (X, Y): [[np.float64(-40.69075096717364), np.float64(7.837103186749864)]]
[np.float64(1.63)]
検出した距離： [407]
角度 1.63 度 のマップからの検出:
  -> Index: 407, Range: 291.30m, X: -40.11m, Y: 8.30m
Detected objects (X, Y): [[np.float64(-40.11331425284652), np.float64(8.296202647305662)]]
[np.float64(1.65)]
検出した距離： [401]
角度 1.65 度 のマップからの検出:
  -> Index: 401, Range: 290.84m, X: -39.65m, Y: 8.36m
Detected objects (X, Y): [[np